# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

# Cleaning, checking the data

In [20]:
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)

In [21]:
df = df.drop(columns=["Unnamed: 0"])

In [22]:
df.columns = ["employment_status" if column == "EmploymentStatus" else column.lower().replace(" ","_") for column in df.columns]

In [23]:
["employment_status" if column == "EmploymentStatus" else column.lower().replace(" ","_") for column in df.columns]

['customer',
 'state',
 'customer_lifetime_value',
 'response',
 'coverage',
 'education',
 'effective_to_date',
 'employment_status',
 'gender',
 'income',
 'location_code',
 'marital_status',
 'monthly_premium_auto',
 'months_since_last_claim',
 'months_since_policy_inception',
 'number_of_open_complaints',
 'number_of_policies',
 'policy_type',
 'policy',
 'renew_offer_type',
 'sales_channel',
 'total_claim_amount',
 'vehicle_class',
 'vehicle_size',
 'vehicle_type']

In [24]:
df.isnull().sum()

customer                            0
state                             631
customer_lifetime_value             0
response                          631
coverage                            0
education                           0
effective_to_date                   0
employment_status                   0
gender                              0
income                              0
location_code                       0
marital_status                      0
monthly_premium_auto                0
months_since_last_claim           633
months_since_policy_inception       0
number_of_open_complaints         633
number_of_policies                  0
policy_type                         0
policy                              0
renew_offer_type                    0
sales_channel                       0
total_claim_amount                  0
vehicle_class                     622
vehicle_size                      622
vehicle_type                     5482
dtype: int64

In [28]:
df.dropna(subset=["response"])

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,48029,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,0,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,22139,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,49078,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,23675,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10904,QI63521,Nevada,19228.463620,No,Basic,High School or Below,2/24/11,Unemployed,M,0,...,0.0,2,Personal Auto,Personal L2,Offer1,Branch,897.600000,Luxury SUV,Medsize,A
10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,1/19/11,Unemployed,F,0,...,NaN,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A
10906,KX53892,Oregon,5259.444853,No,Basic,College,1/6/11,Employed,F,61146,...,0.0,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A
10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2/6/11,Employed,F,39837,...,0.0,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,NaN


# 1
Create a new DataFrame that only includes customers who:
- have a low total_claim_amount (e.g., below $1,000),
- have a response "Yes" to the last marketing campaign.

In [41]:
low_claim_yes_marketing = df[(df["total_claim_amount"] < 1000) & (df["response"] == "Yes")]
low_claim_yes_marketing

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,49078,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
8,FM55990,California,5989.773931,Yes,Premium,College,1/19/11,Employed,M,66839,...,0.0,1,Personal Auto,Personal L1,Offer2,Branch,739.200000,Sports Car,Medsize,NaN
15,CW49887,California,4626.801093,Yes,Basic,Master,1/16/11,Employed,F,79487,...,0.0,1,Special Auto,Special L1,Offer2,Branch,547.200000,SUV,Medsize,NaN
19,NJ54277,California,3746.751625,Yes,Extended,College,2/26/11,Employed,F,41479,...,1.0,1,Personal Auto,Personal L2,Offer2,Call Center,19.575683,Two-Door Car,Large,A
27,MQ68407,Oregon,4376.363592,Yes,Premium,Bachelor,2/28/11,Employed,F,63774,...,0.0,1,Personal Auto,Personal L3,Offer2,Agent,60.036683,Four-Door Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10844,FM31768,Arizona,5979.724161,Yes,Extended,High School or Below,2/7/11,Employed,F,57693,...,0.0,3,Personal Auto,Personal L1,Offer2,Agent,547.200000,Four-Door Car,Medsize,NaN
10852,KZ80424,Washington,8382.478392,Yes,Basic,Bachelor,1/27/11,Employed,M,25998,...,0.0,2,Personal Auto,Personal L2,Offer2,Call Center,791.878042,NaN,NaN,A
10872,XT67997,California,5979.724161,Yes,Extended,High School or Below,2/7/11,Employed,F,57693,...,0.0,3,Personal Auto,Personal L3,Offer2,Agent,547.200000,Four-Door Car,Medsize,NaN
10887,BY78730,Oregon,8879.790017,Yes,Basic,High School or Below,2/3/11,Employed,F,52583,...,0.0,7,Special Auto,Special L2,Offer1,Agent,528.200860,SUV,Small,A


# 2
Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

In [55]:
avg_premium_per_poltype_and_gender = df[df["response"] == "Yes"].groupby(["policy_type", "gender"])["monthly_premium_auto"].mean().round(2)
avg_premium_per_poltype_and_gender

policy_type     gender
Corporate Auto  F         94.30
                M         92.19
Personal Auto   F         99.00
                M         91.09
Special Auto    F         92.31
                M         86.34
Name: monthly_premium_auto, dtype: float64

In [57]:
df["total_claim_amount"].describe()

count    10910.000000
mean       434.888330
std        292.180556
min          0.099007
25%        271.082527
50%        382.564630
75%        547.200000
max       2893.239678
Name: total_claim_amount, dtype: float64

In [58]:
df[df["response"] == "Yes"].groupby(["policy_type", "gender"])["total_claim_amount"].mean().round(2)

policy_type     gender
Corporate Auto  F         433.74
                M         408.58
Personal Auto   F         452.97
                M         457.01
Special Auto    F         453.28
                M         429.53
Name: total_claim_amount, dtype: float64

# 3
Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [91]:
customers_per_state = df.groupby("state")["customer"].count()
customers_per_state

state
Arizona       1937
California    3552
Nevada         993
Oregon        2909
Washington     888
Name: customer, dtype: int64

In [93]:
states_more500customers = df.groupby("state")["customer"].count()[df.groupby("state")["customer"].count() > 500]
states_more500customers

state
Arizona       1937
California    3552
Nevada         993
Oregon        2909
Washington     888
Name: customer, dtype: int64

# 4
Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [100]:
print("Maximum: \n", df.groupby(["education", "gender"])["customer_lifetime_value"].max().round(2))

print("Minimum: \n", df.groupby(["education", "gender"])["customer_lifetime_value"].min().round(2))

print("Median: \n", df.groupby(["education", "gender"])["customer_lifetime_value"].median().round(2))


Maximum: 
 education             gender
Bachelor              F         73225.96
                      M         67907.27
College               F         61850.19
                      M         61134.68
Doctor                F         44856.11
                      M         32677.34
High School or Below  F         55277.45
                      M         83325.38
Master                F         51016.07
                      M         50568.26
Name: customer_lifetime_value, dtype: float64
Minimum: 
 education             gender
Bachelor              F         1904.00
                      M         1898.01
College               F         1898.68
                      M         1918.12
Doctor                F         2395.57
                      M         2267.60
High School or Below  F         2144.92
                      M         1940.98
Master                F         2417.78
                      M         2272.31
Name: customer_lifetime_value, dtype: float64
Median: 
 educatio

# 5
The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [107]:
df["month"] = [value.split("/")[0] for value in df["effective_to_date"]]

In [114]:
policies_per_state_month = pd.DataFrame(df.groupby(["state", "month"])["number_of_policies"].sum())

In [115]:
policies_per_state_month

number_of_policies
state      month                    
Arizona    1                    3052
           2                    2864
California 1                    5673
           2                    4929
Nevada     1                    1493
           2                    1278
Oregon     1                    4697
           2                    3969
Washington 1                    1358
           2                    1225

In [119]:
policies_per_state_month.pivot_table(index="state", columns="month", values=["number_of_policies"])

number_of_policies        
month                       1       2
state                                
Arizona                3052.0  2864.0
California             5673.0  4929.0
Nevada                 1493.0  1278.0
Oregon                 4697.0  3969.0
Washington             1358.0  1225.0

# 6 
Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [165]:
policies_bymonth_bystate = pd.DataFrame(df.groupby(["state", "month"])["number_of_policies"].sum().sort_values(ascending=False)).head(6)
policies_bymonth_bystate

number_of_policies
state      month                    
California 1                    5673
           2                    4929
Oregon     1                    4697
           2                    3969
Arizona    1                    3052
           2                    2864

# 7
The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

In [179]:
df.groupby(["sales_channel"])["response"].value_counts()

sales_channel  response
Agent          No          3148
               Yes          742
Branch         No          2539
               Yes          326
Call Center    No          1792
               Yes          221
Web            No          1334
               Yes          177
Name: count, dtype: int64